In [8]:
# data engineer
import pandas as pd
import numpy as np
from xlrd import open_workbook
import csv
import json
import math
import queue
from pprint import pprint
from sklearn.ensemble import IsolationForest
import datetime
from tqdm import tqdm
FILE_FOLDER = '../prediction_data/%s'
AFTER_FILE_FOLDER = '../prediction_data/afterAnalysis/%s'
def pp_columns(df):pprint(df.columns)

# Generate AirQuality stations information from excel  to csv
### new file name as "aq_stations.csv"

In [22]:
wb = open_workbook(FILE_FOLDER%'Beijing_AirQuality_Stations_en.xlsx')
sheet_names = wb.sheet_names()
sheet = wb.sheet_by_name(sheet_names[0])
number_of_rows = sheet.nrows
number_of_columns = sheet.ncols

stationId = []
longitude = []
latitude = []
stationType = [] 
current_type = ''
for row in range(11, number_of_rows):
    if(sheet.cell(row,0).value == ''):
        continue
    elif(row in [12-1,26-1,39-1,48-1]):
        current_type = sheet.cell(row,0).value
    else:
        stationId.append(sheet.cell(row,0).value)
        longitude.append(sheet.cell(row,1).value)
        latitude.append(sheet.cell(row,2).value)
        stationType.append(current_type)
D = {
    'stationId':pd.Series(stationId),
    'latitude':pd.Series(latitude),
    'longitude':pd.Series(longitude),
    'stationType':pd.Series(stationType),
}
df_aq_station = pd.DataFrame(D)
df_aq_station.head()
# df_aq_station.to_csv(AFTER_FILE_FOLDER%'aq_stations.csv',index=False)

,stationId,latitude,longitude,stationType
0,dongsi_aq,39.929,116.417,Urban Stations
1,tiantan_aq,39.886,116.407,Urban Stations
2,guanyuan_aq,39.929,116.339,Urban Stations
3,wanshouxigong_aq,39.878,116.352,Urban Stations
4,aotizhongxin_aq,39.982,116.397,Urban Stations


# Generate weather stations information from json(from html)  to csv
### new file name as "w_stations.csv"

In [43]:
from pprint import pprint
json_data_file = AFTER_FILE_FOLDER%'meo_data.json'
with open(json_data_file) as f:
    data = json.load(f)

w_station_name = []
w_longitude = []
w_latitude = []
for ele in data['data']:
    w_station_name.append(ele['name'].split(':')[0].replace(' ',''))
    w_longitude.append(ele['lnglat'][0])
    w_latitude.append(ele['lnglat'][1])
    
w_d = {
    'stationId':pd.Series(w_station_name),
    'latitude':pd.Series(w_latitude),
    'longitude':pd.Series(w_longitude)
}
df_observed_weather_stations = pd.DataFrame(w_d)
# df_observed_weather_stations.to_csv(AFTER_FILE_FOLDER%'ob_weather_stations.csv',index=False)
df_observed_weather_stations.head()
#     pprint(ele['lnglat'][0])

,stationId,latitude,longitude
0,shunyi_meo,40.126667,116.615278
1,hadian_meo,39.986944,116.290556
2,yanqing_meo,40.449444,115.968889
3,miyun_meo,40.377500,116.864167
4,huairou_meo,40.357778,116.626944


# loading grid information

In [54]:
file_name = FILE_FOLDER%'Beijing_grid_weather_station.csv'
# df_grid_stations = pd.read_csv(FILE_FOLDER%'Beijing_grid_weather_station.csv')
# df_grid_stations.head()

with open(file_name) as csvfile:
    spamreader = csv.reader(csvfile)
    # print(list(spamreader)
    df_grid_stations = pd.DataFrame(data= list(spamreader),columns=['gridStationId','latitude','longitude'])
csvfile.close()
df_grid_stations.head()
# df_grid_stations.to_csv(AFTER_FILE_FOLDER%'grid_stations.csv',index=False)

# 

In [4]:
df_aq_stations = pd.read_csv(AFTER_FILE_FOLDER%'aq_stations.csv')
df_observed_weather_stations = pd.read_csv(AFTER_FILE_FOLDER%'ob_weather_stations.csv')
df_grid_stations = pd.read_csv(AFTER_FILE_FOLDER%'grid_stations.csv')

# For each air quality station, select n closest observed weather stations and m m grid weather stations.

#### n + m weather information --> manually analysis ( or other scientific methods) --> air quality station weather information
#### air quality station weather information --> neural network model --> pollution results

# Change two queues to one (Select N closest weather stations from observed and grid sations )

In [2]:
def point_dis_square(x1, y1, x2, y2):
    dis = (x1 * 1.0 - x2) * (x1 * 1.0 - x2) + (y1 * 1.0 - y2) * (y1 * 1.0 - y2)
    return math.sqrt(dis)

def get_nearest_stations():
    df_aq_stations = pd.read_csv(AFTER_FILE_FOLDER % 'aq_stations.csv')
    df_observed_weather_stations = pd.read_csv(AFTER_FILE_FOLDER % 'ob_weather_stations.csv')
    df_grid_stations = pd.read_csv(AFTER_FILE_FOLDER % 'grid_stations.csv')

    aq_len = len(df_aq_stations.stationId)
    ow_len = len(df_observed_weather_stations.stationId)
    grid_len = len(df_grid_stations.gridStationId)

    closest_num = 10
    aq_closest_dict = {}

    for i in range(aq_len):

        aq_stationId = df_aq_stations.stationId[i]

        aq_la = df_aq_stations.latitude[i]
        aq_lo = df_aq_stations.longitude[i]
        aq_closest_dict[aq_stationId] = []
        aq_q = queue.PriorityQueue()

        for j in range(grid_len):
            grid_la = df_grid_stations.latitude[j]
            grid_lo = df_grid_stations.longitude[j]
            stationId = df_grid_stations.gridStationId[j]

            y = (point_dis_square(aq_la, aq_lo, grid_la, grid_lo))
            if (aq_q.qsize() < closest_num):
                aq_q.put((-1.0 * y, stationId))
            else:
                if (aq_q.queue[0][0] < -1.0 * y):
                    aq_q.get()
                    aq_q.put((-1.0 * y, stationId))

        for j in range(ow_len):

            ow_la = df_observed_weather_stations.latitude[j]
            ow_lo = df_observed_weather_stations.longitude[j]
            stationId = df_observed_weather_stations.stationId[j]
            y = ((point_dis_square(aq_la, aq_lo, ow_la, ow_lo)))

            if (aq_q.qsize() < closest_num):
                aq_q.put((-1.0 * y, stationId))
            else:
                if (aq_q.queue[0][0] < -1.0 * y):
                    aq_q.get()
                    aq_q.put((-1.0 * y, stationId))

        while (not aq_q.empty()):
            current_ele = aq_q.get()
            aq_closest_dict[aq_stationId].append((-1*current_ele[0],current_ele[1]))
        # pprint(aq_closest_dict[aq_stationId])

    return aq_closest_dict,df_aq_stations

# Generate features for each airQuality stations by N nearest stations

# Pick up related features

In [63]:
def tupleArry_to_list(tupleArry, i_ele):
    if(i_ele == 0):
        w_list = list(map(lambda x:1.0 / x[i_ele],tupleArry))
        return w_list
#         w_sum = sum(w_list)
#         return list(map(lambda x:x*1.0/w_list,w_list))
    return list(map(lambda x:x[i_ele],tupleArry))

def get_mean_without_outliers(df, feature_name):
    df_sub = df.copy()
    pre_mean = df_sub[feature_name].mean()
    """remove abs(num - mean) / mean > 50%"""
    df_sub = df_sub[np.abs((df_sub[feature_name] - df_sub[feature_name].mean())/pre_mean) <= 0.5]
#     print(df_sub)
    df_nearest_3 = df_sub.tail(3)
    
    weight_sum = df_nearest_3['weight'].sum()
    df_nearest_3['weight'] = df_nearest_3['weight'] / weight_sum
    df_nearest_3['after_weighted'] = df_nearest_3['weight'] * df_sub[feature_name]
#     print(df_nearest_3)
#     print(df_nearest_3['after_weighted'].sum())
    return np.around(df_nearest_3['after_weighted'].sum(), decimals=2)

def split_stations(all_stations):
    
    meo_l = []
    grid_l = []
    for i in all_stations:
        if('_meo' in i):
            meo_l.append(i)
        else: grid_l.append(i)
    return meo_l,grid_l
            
    
# def generate_features_for_aqStations(aq_closest_dict,df_grid_stations,df_ow_stations,df_aqLabels):
def generate_features_for_aqStations(aq_closest_dict,df_grid,df_ow):
    starttime = datetime.datetime.now()
    
    df_grid_stations = df_grid.set_index(['station_id'])
    df_ow_stations = df_ow.set_index(['station_id'])
    dict_making = {
        'aq_station_id':[],
        'temperature':[],
        'humidity':[],
        'pressure':[],
        'wind_speed':[],
        'time':[]
    }
    
    for aq_station_id,aq_station_value in tqdm(aq_closest_dict.items(),total=len(aq_closest_dict)):
#     for aq_station_id,aq_station_value in aq_closest_dict.items():
        stationId_list = tupleArry_to_list(aq_station_value, 1)
        weight_dis = tupleArry_to_list(aq_station_value, 0)
        
        ow_s,grid_s = split_stations(stationId_list)
        
        df_grid_temp = df_grid_stations.loc[grid_s].reset_index().set_index(['time'])
        df_ow_temp = df_ow_stations.loc[ow_s].reset_index().set_index(['time'])
        time_span = grid_time_span
        
        grid_time_span = df_grid_stations['time'].unique().tolist()
        
        for t_hour in time_span:
            df_grid_inOneHour = df_grid_temp.loc[t_hour]
            try:
                df_ow_inOneHour = df_ow_temp.loc[t_hour]
            except:
                df_ow_inOneHour = pd.DataFrame(columns=df_ow_temp.columns)
            df_merged = df_grid_inOneHour.append(df_ow_inOneHour)
            df_merged.station_id = df_merged.station_id.astype("category")
            df_merged.station_id.cat.set_categories(stationId_list, inplace=True)
            df_merged = df_merged.sort_values(['station_id']).reset_index()
            df_merged['weight'] = pd.Series(weight_dis)
            dict_making['aq_station_id'].append(aq_station_id)
            for f_name in ['temperature', 'pressure', 'humidity', 'wind_speed']:
                try:
                    new_f = get_mean_without_outliers(df_merged[[f_name,'weight']],f_name)
                    dict_making[f_name].append(new_f)
                except:
                    print(aq_station_id,'at',t_hour,'is NAN')
                    dict_making[f_name].append(np.nan)
            dict_making['time'].append(t_hour)
    endtime = datetime.datetime.now()

    print('cost time:',(endtime - starttime).seconds)
    new_df = pd.DataFrame.from_dict(dict_making)
#     print(new_df)
    return new_df
    
# df_17_whole = generate_features_for_aqStations(aq_nearest_dict,
#                                                 df_grid_17,
#                                                 df_ow_stations_1701_1801)
# df_04 = generate_features_for_aqStations(aq_nearest_dict,df_grid_stations_04,df_ow_stations_04)
# df_04

# Gerate features from 17-(3 to 7) and 18-(3 and 4)

# 17 01 - 18 01

In [4]:
df_grid_stations_1701_1803 = pd.read_csv(FILE_FOLDER%'gridWeather_201701-201803.csv')
df_ow_stations_1701_1801 = pd.read_csv(FILE_FOLDER%'observedWeather_201701-201801.csv')
print(df_ow_stations_1701_1801.columns)
df_ow_stations_1701_1801['datetime'] = pd.to_datetime(df_ow_stations_1701_1801['utc_time'])
df_grid_stations_1701_1803['datetime'] = pd.to_datetime(df_grid_stations_1701_1803['utc_time'])
# df_ow_stations_1701_1801.drop('utc_time',)
# print(len(df_ow_stations_1701_1801))

Index(['station_id', 'longitude', 'latitude', 'utc_time', 'temperature',
       'pressure', 'humidity', 'wind_direction', 'wind_speed', 'weather'],
      dtype='object')


In [14]:
# len(df_grid_stations_1701_1803)
# df_ow_stations_1701_1801
# df_grid_stations_1701_1803[-3:-1]

In [18]:
# start_date = '2017-03-01 00:00:00'
end_date = '2018-02-01 00:00:00'

# start_date_18 = '2018-03-01 00:00:00'
selected_features = ['station_id', 'temperature', 'pressure','humidity', 'wind_speed', 'time', 'datetime']

df_grid_17 = df_grid_stations_1701_1803[df_grid_stations_1701_1803['datetime'] < end_date]
df_ow_stations_1701_1801.columns = ['station_id', 'longitude', 'latitude', 'time', 'temperature',
       'pressure', 'humidity', 'wind_direction', 'wind_speed', 'weather',
       'datetime']
print(df_ow_stations_1701_1801.columns)

df_grid_17.columns = ['station_id', 'longitude', 'latitude', 'time', 'temperature',
       'pressure', 'humidity', 'wind_direction', 'wind_speed', 'datetime']
print(df_grid_17.columns)

df_ow_stations_1701_1801 = df_ow_stations_1701_1801[selected_features]
print(df_ow_stations_1701_1801.columns)

df_grid_17 = df_grid_17[selected_features]
print(df_grid_17.columns)


# df_ow_stations_1703_1707 = df_ow_stations_1701_1801[
#     (df_ow_stations_1701_1801['datetime'] < end_date) &
#     (df_ow_stations_1701_1801['datetime'] >= start_date)
# ]

Index(['station_id', 'longitude', 'latitude', 'time', 'temperature',
       'pressure', 'humidity', 'wind_direction', 'wind_speed', 'weather',
       'datetime'],
      dtype='object')
Index(['station_id', 'longitude', 'latitude', 'time', 'temperature',
       'pressure', 'humidity', 'wind_direction', 'wind_speed', 'datetime'],
      dtype='object')
Index(['station_id', 'temperature', 'pressure', 'humidity', 'wind_speed',
       'time', 'datetime'],
      dtype='object')
Index(['station_id', 'temperature', 'pressure', 'humidity', 'wind_speed',
       'time', 'datetime'],
      dtype='object')


In [57]:
start_date = '2017-03-01 00:00:00'
end_date = '2017-08-01 00:00:00'

start_date_18 = '2018-03-01 00:00:00'

df_grid_stations_1703_1707 = df_grid_stations_1701_1803[
    ((df_grid_stations_1701_1803['datetime'] >= start_date) & 
    (df_grid_stations_1701_1803['datetime'] < end_date))
]

df_ow_stations_1703_1707 = df_ow_stations_1701_1801[
    (df_ow_stations_1701_1801['datetime'] < end_date) &
    (df_ow_stations_1701_1801['datetime'] >= start_date)
]

In [58]:
df_ow_stations_02_03 = pd.read_csv(FILE_FOLDER%'observedWeather_201802-201803.csv')
df_ow_stations_02_03['datetime'] = pd.to_datetime(df_ow_stations_02_03['utc_time'])
df_ow_stations_03 = df_ow_stations_02_03[
   df_ow_stations_02_03['datetime'] >= start_date_18
]
pprint(df_ow_stations_03.head())

df_grid_stations_03 = df_grid_stations_1701_1803[
    df_grid_stations_1701_1803['datetime'] >= start_date_18
]
df_grid_stations_03.head()

      station_id             utc_time      weather  temperature  pressure  \
707  beijing_meo  2018-03-01 00:00:00  Sunny/clear          0.3    1018.6   
708  beijing_meo  2018-03-01 00:00:00  Sunny/clear          0.3    1018.6   
709  beijing_meo  2018-03-01 01:00:00  Sunny/clear          1.9    1018.8   
710  beijing_meo  2018-03-01 02:00:00  Sunny/clear          3.1    1018.3   
711  beijing_meo  2018-03-01 03:00:00  Sunny/clear          4.2    1017.8   

     humidity  wind_speed  wind_direction            datetime  
707        22         1.2              36 2018-03-01 00:00:00  
708        22         1.2              36 2018-03-01 00:00:00  
709        19         2.5              46 2018-03-01 01:00:00  
710        17         2.6              36 2018-03-01 02:00:00  
711        15         2.5              12 2018-03-01 03:00:00  


,stationName,longitude,latitude,utc_time,temperature,pressure,humidity,wind_direction,wind_speed/kph,datetime
6624576,beijing_grid_000,115.0,39.0,2018-03-01 00:00:00,-0.25,981.84,19.90,313.91,2.34,2018-03-01
6624577,beijing_grid_001,115.0,39.1,2018-03-01 00:00:00,-0.51,976.67,19.96,306.14,2.70,2018-03-01
6624578,beijing_grid_002,115.0,39.2,2018-03-01 00:00:00,-1.30,961.15,20.13,292.27,3.95,2018-03-01
6624579,beijing_grid_003,115.0,39.3,2018-03-01 00:00:00,-2.10,945.63,20.30,285.30,5.33,2018-03-01
6624580,beijing_grid_004,115.0,39.4,2018-03-01 00:00:00,-3.48,925.62,20.64,296.62,7.79,2018-03-01


In [59]:
df_ow_stations_04 = pd.read_csv(FILE_FOLDER%'observedWeather_201804.csv')
# print(df_ow_stations_04)
df_ow_stations_04['datetime'] = pd.to_datetime(df_ow_stations_04['time'])
# df_ow_stations_04
df_grid_stations_04 = pd.read_csv(FILE_FOLDER%'gridWeather_201804.csv')
df_grid_stations_04['datetime'] = pd.to_datetime(df_grid_stations_04['time'])
df_grid_stations_04.head()

,id,station_id,time,weather,temperature,pressure,humidity,wind_direction,wind_speed,datetime
0,2027019,beijing_grid_000,2018-04-01 00:00:00,CLEAR_DAY,15.0,974.1536,21.0,346.93,4.18,2018-04-01
1,2027020,beijing_grid_001,2018-04-01 00:00:00,CLEAR_DAY,15.0,960.6752,20.0,329.47,4.50,2018-04-01
2,2027021,beijing_grid_002,2018-04-01 00:00:00,CLEAR_DAY,9.0,947.1968,19.0,315.38,5.16,2018-04-01
3,2027022,beijing_grid_003,2018-04-01 00:00:00,CLEAR_DAY,9.0,931.8720,19.0,307.81,5.74,2018-04-01
4,2027023,beijing_grid_004,2018-04-01 00:00:00,CLEAR_DAY,9.0,914.7008,20.0,304.23,6.08,2018-04-01


In [60]:
# df_ow_stations_1703_1707
# df_grid_stations_1703_1707
# df_ow_stations_03
# df_grid_stations_03
# df_ow_stations_04
# df_grid_stations_04

# df_ow_stations_1703_1707.columns = ['station_id', 'longitude', 'latitude', 'time', 'temperature',
#        'pressure', 'humidity', 'wind_direction', 'wind_speed', 'weather',
#        'datetime']
# print(df_ow_stations_1703_1707.columns)

# df_grid_stations_1703_1707.columns = ['station_id', 'longitude', 'latitude', 'time', 'temperature',
#        'pressure', 'humidity', 'wind_direction', 'wind_speed', 'datetime']
# print(df_grid_stations_1703_1707.columns)

df_ow_stations_03.columns = ['station_id', 'time', 'weather', 'temperature', 'pressure',
       'humidity', 'wind_speed', 'wind_direction', 'datetime']
print(df_ow_stations_03.columns)

df_grid_stations_03.columns = ['station_id', 'longitude', 'latitude', 'time', 'temperature',
       'pressure', 'humidity', 'wind_direction', 'wind_speed', 'datetime']
print(df_grid_stations_03.columns)

df_ow_stations_04.columns = ['id', 'station_id', 'time', 'weather', 'temperature', 'pressure',
       'humidity', 'wind_speed', 'wind_direction', 'datetime']
df_ow_stations_04.drop('id',axis=1,inplace=True)
print(df_ow_stations_04.columns)

df_grid_stations_04.columns = ['id', 'station_id', 'time', 'weather', 'temperature', 'pressure',
       'humidity', 'wind_direction', 'wind_speed', 'datetime']
df_grid_stations_04.drop('id',axis=1,inplace=True)
print(df_grid_stations_04.columns)

Index(['station_id', 'time', 'weather', 'temperature', 'pressure', 'humidity',
       'wind_speed', 'wind_direction', 'datetime'],
      dtype='object')
Index(['station_id', 'longitude', 'latitude', 'time', 'temperature',
       'pressure', 'humidity', 'wind_direction', 'wind_speed', 'datetime'],
      dtype='object')
Index(['station_id', 'time', 'weather', 'temperature', 'pressure', 'humidity',
       'wind_speed', 'wind_direction', 'datetime'],
      dtype='object')
Index(['station_id', 'time', 'weather', 'temperature', 'pressure', 'humidity',
       'wind_direction', 'wind_speed', 'datetime'],
      dtype='object')


In [61]:
selected_features = ['station_id', 'temperature', 'pressure','humidity', 'wind_speed', 'time', 'datetime']

# df_ow_stations_1703_1707 = df_ow_stations_1703_1707[selected_features]
# print(df_ow_stations_1703_1707.columns)

# df_grid_stations_1703_1707 = df_grid_stations_1703_1707[selected_features]
# print(df_grid_stations_1703_1707.columns)

df_ow_stations_03 = df_ow_stations_03[selected_features]
print(df_ow_stations_03.columns)

df_grid_stations_03 = df_grid_stations_03[selected_features]
print(df_grid_stations_03.columns)

df_ow_stations_04 = df_ow_stations_04[selected_features]
print(df_ow_stations_04.columns)

df_grid_stations_04 = df_grid_stations_04[selected_features]
print(df_grid_stations_04.columns)

Index(['station_id', 'temperature', 'pressure', 'humidity', 'wind_speed',
       'time', 'datetime'],
      dtype='object')
Index(['station_id', 'temperature', 'pressure', 'humidity', 'wind_speed',
       'time', 'datetime'],
      dtype='object')
Index(['station_id', 'temperature', 'pressure', 'humidity', 'wind_speed',
       'time', 'datetime'],
      dtype='object')
Index(['station_id', 'temperature', 'pressure', 'humidity', 'wind_speed',
       'time', 'datetime'],
      dtype='object')


In [24]:
aq_nearest_dict, df_aq_stations = get_nearest_stations()
aq_nearest_dict

{'dongsi_aq': [(0.13908898021869456, 'hadian_meo'),
  (0.13685759021698704, 'beijing_grid_283'),
  (0.1336117299371322, 'beijing_meo'),
  (0.1301153334545976, 'beijing_grid_302'),
  (0.12054044964243844, 'beijing_grid_282'),
  (0.10922453936730264, 'beijing_grid_325'),
  (0.0879204185613327, 'beijing_grid_324'),
  (0.08706475859316841, 'chaoyang_meo'),
  (0.07300684899377298, 'beijing_grid_304'),
  (0.03361547262794414, 'beijing_grid_303')],
 'tiantan_aq': [(0.14712239802286733, 'beijing_grid_325'),
  (0.13727709204380256, 'beijing_grid_281'),
  (0.12666885963013072, 'beijing_grid_323'),
  (0.11500842659946843, 'chaoyang_meo'),
  (0.11421471008587752, 'beijing_grid_304'),
  (0.10791200118616547, 'beijing_grid_282'),
  (0.10139791830600552, 'beijing_meo'),
  (0.09404786015640172, 'beijing_grid_324'),
  (0.08628441342444078, 'beijing_grid_302'),
  (0.015652475842503358, 'beijing_grid_303')],
 'guanyuan_aq': [(0.14269547995645174, 'beijing_grid_302'),
  (0.1419929575718424, 'beijing_grid_

In [25]:
selectef_labels = ['station_id','time','PM25_Concentration','PM10_Concentration','O3_Concentration']
re_named_labels = ['station_id','time','PM25','PM10','O3']

# df_ow_stations_1703_1707
# df_grid_stations_1703_1707

# df_ow_stations_03
# df_grid_stations_03

# df_ow_stations_04
# df_grid_stations_04
aq_04 = pd.read_csv(FILE_FOLDER % 'aiqQuality_201804.csv')
aq_04 = aq_04[selectef_labels]
aq_04.columns = re_named_labels

aq_04.head()

,station_id,time,PM25,PM10,O3
0,dongsi_aq,2018-04-01 02:00:00,259.0,NaN,56.0
1,tiantan_aq,2018-04-01 02:00:00,250.0,NaN,64.0
2,guanyuan_aq,2018-04-01 02:00:00,240.0,246.0,49.0
3,wanshouxigong_aq,2018-04-01 02:00:00,255.0,260.0,65.0
4,aotizhongxin_aq,2018-04-01 02:00:00,266.0,NaN,45.0


In [12]:
aq_03 = pd.read_csv(FILE_FOLDER % 'airQuality_201802-201803.csv')
print(aq_03.columns)
aq_03 = aq_03[['stationId', 'utc_time','PM2.5','PM10','O3']]
aq_03.columns = re_named_labels
aq_03['datetime'] = pd.to_datetime(aq_03['time'])
aq_03 = aq_03[
    aq_03['datetime']>= start_date_18
]
aq_03 = aq_03[re_named_labels]
aq_03.head()

Index(['stationId', 'utc_time', 'PM2.5', 'PM10', 'NO2', 'CO', 'O3', 'SO2'], dtype='object')


,station_id,time,PM25,PM10,O3
23388,aotizhongxin_aq,2018-03-01 00:00:00,9.0,47.0,58.0
23389,aotizhongxin_aq,2018-03-01 01:00:00,14.0,47.0,61.0
23390,aotizhongxin_aq,2018-03-01 02:00:00,11.0,38.0,61.0
23391,aotizhongxin_aq,2018-03-01 03:00:00,4.0,33.0,NaN
23392,aotizhongxin_aq,2018-03-01 04:00:00,NaN,18.0,82.0


In [19]:
aq_1703_1707 = pd.read_csv(FILE_FOLDER % 'airQuality_201701-201801.csv')
len(aq_1703_1707)
aq_1703_1707

# aq_1703_1707 = aq_1703_1707[['stationId', 'utc_time','PM2.5','PM10','O3']]
# aq_1703_1707.columns = re_named_labels
# aq_1703_1707['datetime'] = pd.to_datetime(aq_1703_1707['time'])
# aq_1703_1707 = aq_1703_1707[
#     (aq_1703_1707['datetime'] >= start_date) &
#     (aq_1703_1707['datetime']< end_date) 
# ]
# # aq_1703_1707 = aq_1703_1707[re_named_labels]
# aq_1703_1707.head()

311010

In [13]:
# df_03 = generate_features_for_aqStations(aq_nearest_dict,df_grid_stations_03,df_ow_stations_03,aq_03)

  0%|          | 0/35 [00:00<?, ?it/s]/Users/haomao/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/haomao/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|██████████| 35/35 [1:55:06<00:00, 203.31s/it]

cost time: 6906


In [32]:
splited_date = '2017-05-01 00:00:00'
# aq_1703_1704 = aq_1703_1707[
#     (aq_1703_1707['datetime'] < splited_date) 
# ]

aq_1705_1707 =  aq_1703_1707[
    aq_1703_1707['datetime'] >= splited_date
]
aq_1705_1707 = aq_1705_1707[re_named_labels]
df_ow_stations_1705_1707 = df_ow_stations_1703_1707[
    df_ow_stations_1703_1707['datetime'] >= splited_date
]
df_grid_stations_1705_1707= df_grid_stations_1703_1707[
    df_grid_stations_1703_1707['datetime'] >= splited_date
]
aq_1705_1707.head()

,station_id,time,PM25,PM10,O3
2662,aotizhongxin_aq,2017-05-01 00:00:00,58.0,114.0,3.0
2663,aotizhongxin_aq,2017-05-01 01:00:00,54.0,116.0,24.0
2664,aotizhongxin_aq,2017-05-01 02:00:00,66.0,84.0,36.0
2665,aotizhongxin_aq,2017-05-01 03:00:00,27.0,93.0,59.0
2666,aotizhongxin_aq,2017-05-01 04:00:00,24.0,85.0,75.0


In [35]:
# df_1705_1707 = generate_features_for_aqStations(aq_nearest_dict,
#                                                 df_grid_stations_1705_1707,
#                                                 df_ow_stations_1705_1707,
#                                                 aq_1705_1707)

  0%|          | 0/35 [00:00<?, ?it/s]/Users/haomao/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['hadian_meo', 'beijing_grid_283', 'beijing_meo', 'beijing_grid_302', 'beijing_grid_282', 'beijing_grid_325', 'beijing_grid_324', 'chaoyang_meo', 'beijing_grid_304', 'beijing_grid_303']


/Users/haomao/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  3%|▎         | 1/35 [08:22<4:44:54, 502.78s/it]

['beijing_grid_325', 'beijing_grid_281', 'beijing_grid_323', 'chaoyang_meo', 'beijing_grid_304', 'beijing_grid_282', 'beijing_meo', 'beijing_grid_324', 'beijing_grid_302', 'beijing_grid_303']


  6%|▌         | 2/35 [16:34<4:34:45, 499.58s/it]

...

['beijing_grid_304', 'beijing_grid_323', 'beijing_grid_324', 'daxing_meo', 'fengtai_meo', 'beijing_meo', 'beijing_grid_281', 'beijing_grid_282', 'beijing_grid_302', 'beijing_grid_303']


 97%|█████████▋| 34/35 [5:11:19<10:04, 604.12s/it]

['beijing_grid_302', 'beijing_grid_323', 'beijing_meo', 'beijing_grid_346', 'beijing_grid_345', 'beijing_grid_304', 'beijing_grid_303', 'beijing_grid_325', 'beijing_grid_324', 'chaoyang_meo']


100%|██████████| 35/35 [5:21:15<00:00, 601.42s/it]

cost time: 19275


In [30]:
splited_date = '2017-05-01 00:00:00'
# aq_1703_1704 = aq_1703_1707[
#     (aq_1703_1707['datetime'] < splited_date) 
# ]

aq_1703_1704 =  aq_1703_1707[
    aq_1703_1707['datetime'] < splited_date
]
aq_1703_1704 = aq_1703_1704[re_named_labels]

df_ow_stations_1703_1704 = df_ow_stations_1703_1707[
    df_ow_stations_1703_1707['datetime'] < splited_date
]
df_grid_stations_1703_1704= df_grid_stations_1703_1707[
    df_grid_stations_1703_1707['datetime'] < splited_date
]
aq_1703_1704.head()

,station_id,time,PM25,PM10,O3
1329,aotizhongxin_aq,2017-03-01 00:00:00,9.0,43.0,96.0
1330,aotizhongxin_aq,2017-03-01 01:00:00,4.0,54.0,100.0
1331,aotizhongxin_aq,2017-03-01 02:00:00,7.0,75.0,109.0
1332,aotizhongxin_aq,2017-03-01 03:00:00,7.0,135.0,118.0
1333,aotizhongxin_aq,2017-03-01 04:00:00,8.0,58.0,120.0


In [31]:
df_1703_1704 = generate_features_for_aqStations(aq_nearest_dict,
                                                df_grid_stations_1703_1704,
                                                df_ow_stations_1703_1704,
                                                aq_1703_1704)

  0%|          | 0/35 [00:00<?, ?it/s]/Users/haomao/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/haomao/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|██████████| 35/35 [3:48:29<00:00, 409.18s/it]  

cost time: 13709


In [32]:
# df_1703_1704.to_csv('f_1703_1704.csv')

# Merge all train data

In [33]:
df_1703_1704 = pd.read_csv(AFTER_FILE_FOLDER%'f_1703_1704.csv')
df_1705_1707 = pd.read_csv(AFTER_FILE_FOLDER%'f_1705_1707.csv')
df_1803 = pd.read_csv(AFTER_FILE_FOLDER%'f_03.csv')
df_1804 = pd.read_csv(AFTER_FILE_FOLDER%'f_04.csv')

In [39]:
print(len(df_1703_1704))
df_train = df_1703_1704.append(df_1705_1707)
print(len(df_train))
df_train = df_train.append(df_1803)
print(len(df_train))
df_train = df_train.append(df_1804)
print(len(df_train))
df_train = df_train[['aq_station_id', 'temperature', 'humidity', 'pressure',
       'wind_speed', 'time', 'PM25', 'PM10', 'O3']]
df_train.to_csv(AFTER_FILE_FOLDER%'train_data.csv',index=False)

44975
104510
130058
153160


# Generate the test data for each aq station

In [8]:
aq_nearest_dict, df_aq_stations = get_nearest_stations()

In [9]:
def tupleArry_to_list(tupleArry, i_ele):
    if(i_ele == 0):
        w_list = list(map(lambda x:1.0 / x[i_ele],tupleArry))
        return w_list
    return list(map(lambda x:x[i_ele],tupleArry))

def get_mean_without_outliers(df, feature_name):
    df_sub = df.copy()
    pre_mean = df_sub[feature_name].mean()
    """remove abs(num - mean) / mean > 50%"""
    df_sub = df_sub[np.abs((df_sub[feature_name] - df_sub[feature_name].mean())/pre_mean) <= 0.5]
#     print(df_sub)
    df_nearest_3 = df_sub.tail(3)
    
    weight_sum = df_nearest_3['weight'].sum()
    df_nearest_3['weight'] = df_nearest_3['weight'] / weight_sum
    df_nearest_3['after_weighted'] = df_nearest_3['weight'] * df_sub[feature_name]
#     print(df_nearest_3)
#     print(df_nearest_3['after_weighted'].sum())
    return np.around(df_nearest_3['after_weighted'].sum(), decimals=2)

def generate_features_for_aqStations_testData(aq_closest_dict,df_grid_stations,df_ow_stations):
    starttime = datetime.datetime.now()
    dict_making = {
        'aq_station_id':[],
        'temperature':[],
        'humidity':[],
        'pressure':[],
        'wind_speed':[],
        'time':[],
    }
    for aq_station_id,aq_station_value in tqdm(aq_closest_dict.items(),total=len(aq_closest_dict)):
#         grid_value = aq_closest_dict[aq_station_id]
        stationId_list = tupleArry_to_list(aq_station_value, 1)
        weight_dis = tupleArry_to_list(aq_station_value, 0)
        #     dis_list = tupleArry_to_list(grid_value,0)
        #     weight_s = pd.Series(dis_list)
#         print(stationId_list)
        df_grid_temp = df_grid_stations[df_grid_stations['station_id'].isin(stationId_list)]
        df_ow_temp = df_ow_stations[df_ow_stations['station_id'].isin(stationId_list)]
        #     print(df_ow_temp)
        grid_time_span = df_grid_temp['time'].unique().tolist()
        ow_time_span = df_ow_temp['time'].unique().tolist()
#         print('time',ow_time_span)
        time_span = grid_time_span if len(grid_time_span) > len(ow_time_span) else ow_time_span

        for t_hour in time_span:
            df_grid_inOneHour = df_grid_temp[df_grid_temp['time'] == t_hour]
            df_ow_inOneHour = df_ow_temp[df_ow_temp['time'] == t_hour]
            
            df_merged = df_grid_inOneHour.append(df_ow_inOneHour)
            df_merged.station_id = df_merged.station_id.astype("category")
            df_merged.station_id.cat.set_categories(stationId_list, inplace=True)
#             pprint(stationId_list)
            df_merged = df_merged.sort_values(['station_id']).reset_index()
            df_merged['weight'] = pd.Series(weight_dis)
            
            dict_making['aq_station_id'].append(aq_station_id)
            for f_name in ['temperature', 'pressure', 'humidity', 'wind_speed']:
                try:
                    new_f = get_mean_without_outliers(df_merged[[f_name,'weight']],f_name)
                    dict_making[f_name].append(new_f)
                except:
                    print(aq_station_id,'at',t_hour,'is NAN')
                    dict_making[f_name].append(np.nan)
                
            dict_making['time'].append(t_hour)
    endtime = datetime.datetime.now()

    print('cost time:',(endtime - starttime).seconds)
    new_df = pd.DataFrame.from_dict(dict_making)
    
    return new_df

In [10]:
df_ow_050102 = pd.read_csv(FILE_FOLDER%'observedWeather_20180501-20180502.csv')
df_grid_050102 = pd.read_csv(FILE_FOLDER%'gridWeather_20180501-20180502.csv')

df_ow_050102 = df_ow_050102[['station_id', 'time', 'weather', 'temperature', 'pressure',
       'humidity', 'wind_speed']]
df_grid_050102 = df_grid_050102[['station_id', 'time', 'weather', 'temperature', 'pressure',
       'humidity', 'wind_speed']]

In [11]:
print(df_ow_050102.columns)
print(df_grid_050102.columns)

Index(['station_id', 'time', 'weather', 'temperature', 'pressure', 'humidity',
       'wind_speed'],
      dtype='object')
Index(['station_id', 'time', 'weather', 'temperature', 'pressure', 'humidity',
       'wind_speed'],
      dtype='object')


In [12]:
df_test = generate_features_for_aqStations_testData(aq_nearest_dict,df_grid_050102,df_ow_050102)

  0%|          | 0/35 [00:00<?, ?it/s]/Users/haomao/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/haomao/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|██████████| 35/35 [06:12<00:00, 11.06s/it]

cost time: 372


In [13]:
df_test.to_csv(AFTER_FILE_FOLDER%'test_data.csv',index=False)

In [70]:
pd.date_range(start='2017-01-01 00:00:00', end='2018-01-01 00:00:00', closed=None, freq='H')

DatetimeIndex(['2017-01-01 00:00:00', '2017-01-01 01:00:00',
               '2017-01-01 02:00:00', '2017-01-01 03:00:00',
               '2017-01-01 04:00:00', '2017-01-01 05:00:00',
               '2017-01-01 06:00:00', '2017-01-01 07:00:00',
               '2017-01-01 08:00:00', '2017-01-01 09:00:00',
               ...
               '2017-12-31 15:00:00', '2017-12-31 16:00:00',
               '2017-12-31 17:00:00', '2017-12-31 18:00:00',
               '2017-12-31 19:00:00', '2017-12-31 20:00:00',
               '2017-12-31 21:00:00', '2017-12-31 22:00:00',
               '2017-12-31 23:00:00', '2018-01-01 00:00:00'],
              dtype='datetime64[ns]', length=8761, freq='H')

# Join all tables

In [39]:
# AFTER_FILE_FOLDER
df_weather_17 = pd.read_csv(AFTER_FILE_FOLDER % 'new_f2017.csv')
df_weather_18_0103 = pd.read_csv(AFTER_FILE_FOLDER % 'new_f0103_no_missing.csv')
df_weather_19_04 = pd.read_csv(AFTER_FILE_FOLDER % 'new_f04_no_missing.csv')
pp_columns(df_weather_17)
pp_columns(df_weather_18_0103)
pp_columns(df_weather_19_04)
df_weather = df_weather_17[['aq_station_id', 'temperature', 'humidity', 'pressure',
       'wind_speed', 'time']].append(df_weather_18_0103,sort=False).append(df_weather_19_04,sort=False)

Index(['Unnamed: 0', 'aq_station_id', 'temperature', 'humidity', 'pressure',
       'wind_speed', 'time'],
      dtype='object')
Index(['aq_station_id', 'time', 'temperature', 'humidity', 'pressure',
       'wind_speed'],
      dtype='object')
Index(['aq_station_id', 'time', 'temperature', 'humidity', 'pressure',
       'wind_speed'],
      dtype='object')


In [40]:
df_weather = df_weather.rename(columns={'aq_station_id':'station_id'}).set_index(['station_id','time'])

In [41]:
df_aq17_1803 = pd.read_csv(AFTER_FILE_FOLDER % 'aq_by_time_17_1803.csv')
df_aq1804 = pd.read_csv(AFTER_FILE_FOLDER % 'aq_by_time01-04.csv')
# ['station_id', 'time', 'PM25', 'PM10', 'O3']

In [42]:
df_aq_all = df_aq17_1803[['station_id', 'time', 'PM25', 'PM10', 'O3']].\
                                            append(df_aq1804,sort=False).\
                                            set_index(['station_id','time'])
df_aq_all = df_aq_all[~df_aq_all.index.duplicated(keep='first')]
# df_aq_all = df_aq_all.reset_index()

In [46]:
df_aq_with_weather = df_weather.join(df_aq_all,on=('station_id', 'time')).reset_index()

In [49]:
df_aq_with_weather.to_csv(AFTER_FILE_FOLDER % 'aq_with_weather.csv',index=False)